In [ ]:
# Внимание!!! Важно, что бы файлы с данными и исполняемый файл находились в одной папке, 
# тогда пути к тестовым и тренировочным наборам будут содержать только имена файлов.
# 
# В пути к тренировочным и тестовым данным запрежается использовать абсалютную адресацию, 
# то есть адресацию, в которой присутствуют имена папок. Путь должен содержать только имя файла.
#
# Напоминание: под моделью машинного обучения понимаются все действия с исходными данными, 
# которые необходимо произвести, что бы сопоставить признаки целевому значению.

### Область работы 1 (библиотеки)

In [ ]:
# Данный блок в области 1 выполняется преподавателем
# 
# данный блок предназначен только для подключения необходимых библиотек
# запрещается подключать библиотеки в других блоках
#
# установка дополнительных библиотек размещается прямо здесь (обязательно закоментированы)
#
# pip install

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt
#pip install category_encoders
import category_encoders as ce

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split , GridSearchCV , cross_val_score , RandomizedSearchCV,KFold,RepeatedKFold
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import OneHotEncoder , LabelEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.linear_model import ElasticNet, LinearRegression, Lasso, Ridge, SGDRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import cross_validate
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, Normalizer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, KFold
from sklearn import svm
from sklearn.model_selection import learning_curve
from matplotlib.colors import ListedColormap
from sklearn.ensemble import RandomForestRegressor

### Область работы 2 (поиск лучшей модели)

In [ ]:
# Данный блок(и) НЕ выполняются преподавателем в области 2
# блок(и) предназначены для поиска лучшей модели
# 
# Запрещается размещать данные блоки за пределами обасти 2
# Все блоки данной области должны быть выполнены
#
# Путь к тренировочному набору
# 

path_train = pd.read_csv('dim_train.csv') # содержит только имя файла, без имен папок !!!
df = pd.DataFrame(path_train)
display(df.head(),df.shape)

# БЛОК 2  с моделями в отдельном файле

### Область работы 3 (выполнение лучшей модели)

In [ ]:
# Данный блок(и) в области 3 выполняется преподавателем
#
# В области находится одна, единственная, итоговая модель машинного обучения с однозначными, 
# зафиксированными параметрами
#
# В данной области категорически запрещается искать, выбирать, улучшать, оптимизировать, 
# тюниговать и т.д. модель машинного обучения

In [ ]:
path_train = pd.read_csv('dim_train.csv') 
df = pd.DataFrame(path_train)
display(df.head(),df.shape)

path_test = pd.read_csv('dim_test.csv')
df1 = pd.DataFrame(path_test)
display(df1.head(),df1.shape)

In [ ]:
df = df.drop(df[df['y'] == 0].index)
df = df.drop(df[df['x'] == 0].index)
df = df.drop(df[df['z'] == 0].index)
df.shape

In [ ]:
df = df[df['x'] < 30 ]
df = df[df['y'] < 30 ]
df = df[(df['z'] < 30) & (df['z'] > 2)]
df = df[(df['depth'] < 75) & (df['depth'] > 50) ]
df = df[(df['table'] < 80) & (df['table'] > 45) ]
df.shape

In [ ]:
cat_features = ['cut','color']
axis_features = ['x','y','z']
num_features = ['carat','depth','table']
y = np.array(df.price)
X = df.drop(columns=['price'])

In [ ]:
clarity_map = [{
    'col':'clarity',
    'mapping':{'IF':7, 'VVS1':6, 'VVS2':5, 'VS1':4, 'VS2':3, 
               'SI1':2, 'SI2':1, 'I1':0}
    }]

clarity_transformer = Pipeline(steps=[
    ('ce',ce.OrdinalEncoder(mapping = clarity_map))
    ])

# для деревьев можно не стандартизировать данные в фичах 
CT = ColumnTransformer(transformers=[("axis", 'passthrough', axis_features),
                                               ('num', 'passthrough', num_features),
                                               ('cat', OneHotEncoder(), cat_features),
                                               ("ordinal_map", clarity_transformer, ['clarity'])])
display(CT)

In [ ]:
# есть 0 значения => убираем их
df1.describe()

In [ ]:
# тут есть 2 нулевых значения
df1 = df1.drop(df1[df1['y'] == 0].index)
df1 = df1.drop(df1[df1['x'] == 0].index)
df1 = df1.drop(df1[df1['z'] == 0].index)
df1.shape

In [ ]:
# делаем маску
mask = df1.index
#pd.DataFrame(mask)

In [ ]:
# Блок(и) обучения и поверки модели

In [ ]:
finish_pipe =  Pipeline([
            ('preprocessing', CT), 
            ('estimator',RandomForestRegressor(n_estimators = 400, min_samples_leaf = 1 , max_features = 20))
            ])

finish_pipe.fit(X, y)

print(finish_pipe.score(X,y))

In [ ]:
# Блок предсказания с использованием тестового набора

In [ ]:
y_predict = finish_pipe.predict(df1)
y_predict.shape

In [ ]:
# Маска отобраных номеров строк для предсказания из тестового набора
mask = mask 
# Название вектора предсказанных значений  y_predict полученого на основании тестового набора
y_predict = y_predict